In [1]:
import pandas as pd
import requests
from urllib.parse import urlencode

In [2]:
# Чтение данных из облака
def download_link(public_key):

    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    return download_url

In [3]:
df = pd.read_csv(download_link('https://disk.yandex.ru/d/DnyWMTr6L6dENg'), parse_dates=['date'])

In [4]:
df.dtypes

date            datetime64[ns]
event                   object
purchase_sum           float64
os_name                 object
device_id                int64
gender                  object
city                    object
utm_source              object
dtype: object

MAU февраля:

In [5]:
df['month'] = df['date'].apply(lambda x: x.month)

In [6]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,1
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,1
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,1
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,1
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,1


In [7]:
df[df['month'] == 2].groupby('month', as_index=False)\
    .nunique()

,month,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2,29,7,4358,2,75032,2,2,7


Количество установок в январе:

In [8]:
df.query("month == 1 & event == 'app_install'").device_id.nunique()

80297

In [9]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,1
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,1
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,1
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,1
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,1


С какого платного маркетингового канала пришло больше всего новых пользователей?

In [13]:
df.groupby('utm_source', as_index=False)\
    .agg({'device_id': 'nunique'})\
    .sort_values('device_id', ascending=False)

,utm_source,device_id
0,-,52273
6,yandex-direct,40712
2,google_ads,38096
5,vk_ads,34086
3,instagram_ads,31048
1,facebook_ads,25959
4,referal,15926


Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?


In [14]:
df.groupby('event', as_index=False)\
    .agg(unique_users=('device_id', 'nunique'))\
    .sort_values('unique_users', ascending=False)

,event,unique_users
1,app_start,190884
5,search,184488
2,choose_item,155691
0,app_install,154597
6,tap_basket,125414
4,register,78310
3,purchase,70875
